# Fake News Detection Using Natural Language Processing and Machine Learning

### Setup and imports

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
import string


In [2]:
fake_news_train = pd.read_csv("train.tsv", sep="\t")
fake_news_valid= pd.read_csv('valid.tsv', sep='\t')
fake_news_test = pd.read_csv('test.tsv', sep='\t')


In [4]:
fake_news_train.head()

,Statement Id,Label,Statement,Subject,Speaker,Speaker's Job,State,Party,Barely True Counts,False Count,Half True Counts,Mostly True Counts,Pants on Fire Counts,Context Venue
0,2635.json,FALSE,Says the Annies List political group supports ...,abortion,dwayne-bohac,State representative,Texas,republican,0.0,1.0,0.0,0.0,0.0,a mailer
1,10540.json,half-true,When did the decline of coal start? It started...,"energy,history,job-accomplishments",scott-surovell,State delegate,Virginia,democrat,0.0,0.0,1.0,1.0,0.0,a floor speech.
2,324.json,mostly-true,"Hillary Clinton agrees with John McCain ""by vo...",foreign-policy,barack-obama,President,Illinois,democrat,70.0,71.0,160.0,163.0,9.0,Denver
3,1123.json,FALSE,Health care reform legislation is likely to ma...,health-care,blog-posting,NaN,NaN,none,7.0,19.0,3.0,5.0,44.0,a news release
4,9028.json,half-true,The economic turnaround started at the end of ...,"economy,jobs",charlie-crist,NaN,Florida,democrat,15.0,9.0,20.0,19.0,2.0,an interview on CNN


In [5]:
fake_news_train.tail()

,Statement Id,Label,Statement,Subject,Speaker,Speaker's Job,State,Party,Barely True Counts,False Count,Half True Counts,Mostly True Counts,Pants on Fire Counts,Context Venue
10264,5473.json,mostly-true,There are a larger number of shark attacks in ...,"animals,elections",aclu-florida,NaN,Florida,none,0.0,1.0,1.0,1.0,0.0,"interview on ""The Colbert Report"""
10265,3408.json,mostly-true,Democrats have now become the party of the [At...,elections,alan-powell,NaN,Georgia,republican,0.0,0.0,0.0,1.0,0.0,an interview
10266,3959.json,half-true,Says an alternative to Social Security that op...,"retirement,social-security",herman-cain,NaN,Georgia,republican,4.0,11.0,5.0,3.0,3.0,a Republican presidential debate
10267,2253.json,FALSE,On lifting the U.S. Cuban embargo and allowing...,"florida,foreign-policy",jeff-greene,NaN,Florida,democrat,3.0,1.0,3.0,0.0,0.0,a televised debate on Miami's WPLG-10 against ...
10268,1155.json,pants-fire,The Department of Veterans Affairs has a manua...,"health-care,veterans",michael-steele,chairman of the Republican National Committee,Maryland,republican,0.0,1.0,1.0,0.0,2.0,a Fox News interview


In [6]:
fake_news_train.describe()

,Barely True Counts,False Count,Half True Counts,Mostly True Counts,Pants on Fire Counts
count,10263.000000,10263.000000,10263.000000,10263.000000,10263.000000
mean,11.549839,13.304394,17.168762,16.465166,6.202670
std,18.992224,24.132578,35.899983,36.202646,16.121555
min,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,2.000000,3.000000,3.000000,1.000000
75%,12.000000,15.000000,13.000000,11.000000,5.000000
max,70.000000,114.000000,160.000000,163.000000,105.000000


In [7]:
fake_news_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10269 entries, 0 to 10268
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Statement Id          10269 non-null  object 
 1   Label                 10269 non-null  object 
 2   Statement             10269 non-null  object 
 3   Subject               10263 non-null  object 
 4   Speaker               10263 non-null  object 
 5   Speaker's Job         7362 non-null   object 
 6   State                 8053 non-null   object 
 7   Party                 10263 non-null  object 
 8   Barely True Counts    10263 non-null  float64
 9   False Count           10263 non-null  float64
 10  Half True Counts      10263 non-null  float64
 11  Mostly True Counts    10263 non-null  float64
 12  Pants on Fire Counts  10263 non-null  float64
 13  Context Venue         10163 non-null  object 
dtypes: float64(5), object(9)
memory usage: 1.1+ MB


### Data Preprocessing

#### Remove duplicates

#### Normalize text: lowercase, punctuation removal

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@w+|\#','', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text

fake_news_train['clean_text'] = fake_news_train['text'].apply(clean_text)
fake_news_valid['clean_text'] = fake_news_valid['text'].apply(clean_text)
fake_news_test['clean_text'] = fake_news_test['text'].apply(clean_text)